This script is to understand the word embedding and how it works.

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import torch.nn.functional as f

In [2]:
checkpoint = "D:\\LLM\\thesisPractical\\fine_tuned_models\\Qwen2.5-3B-Instruct-software-model_completion"

In [3]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [16]:
model = AutoModel.from_pretrained(
    checkpoint, device_map="auto", quantization_config=quantization_config
)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
vocab = tokenizer.get_vocab()
print("Vocabulary size:", len(vocab))

Vocabulary size: 151665


In [54]:
text1 = "wait"
text2 = "de"
inputs1 = tokenizer(text1, return_tensors="pt", padding=True, truncation=True)
inputs2 = tokenizer(text2, return_tensors="pt", padding=True, truncation=True)
input_ids1 = inputs1['input_ids'].to('cuda')
attention_mask1 = inputs1['attention_mask'].to('cuda')
input_ids2 = inputs1['input_ids'].to('cuda')
attention_mask2= inputs1['attention_mask'].to('cuda')

# Get embeddings from the model
with torch.no_grad():
    outputs1 = model(**inputs1, output_hidden_states=True)
    outputs2 = model(input_ids=input_ids2, attention_mask=attention_mask2, output_hidden_states=True)

# Use CLS token embeddings (first token in the sequence)
embedding1 = outputs1.last_hidden_state[:, 0, :]
embedding2 = outputs2.last_hidden_state[:, 0, :]
print(outputs1)

BaseModelOutputWithPast(last_hidden_state=tensor([[[ 2.2305, -0.8589,  2.2832,  ..., -1.3838, -0.5322, -0.3289]]],
       dtype=torch.float16), past_key_values=((tensor([[[[-3.9141e+00, -1.0303e+00,  1.7227e+00,  2.4941e+00, -3.6055e+00,
           -1.0049e+00,  2.9663e-01,  2.4512e+00, -9.5508e-01, -9.4824e-01,
           -4.1328e+00,  7.0703e-01,  2.6250e+00, -2.6489e-01, -9.0820e-01,
           -1.5273e+00,  4.0430e-01,  0.0000e+00, -4.7461e-01,  1.1973e+00,
            2.8828e+00, -2.8122e-02, -4.4604e-01, -5.5542e-03, -1.6052e-01,
           -8.0566e-02,  6.6309e-01, -2.8203e+00, -7.1045e-02, -2.3730e-01,
            5.7764e-01, -6.7871e-01,  1.4417e-01, -7.8418e-01, -4.5898e-02,
            5.8545e-01, -4.1553e-01, -5.0195e-01, -4.4336e+00,  9.9316e-01,
           -1.7432e-01,  1.2622e-01,  1.3535e+00, -7.4902e-01,  9.9951e-01,
           -1.6260e-01, -8.5547e-01, -6.7480e-01,  1.1660e+00, -1.7266e+01,
           -4.5156e+00, -1.4629e+00,  6.9062e+00, -7.5273e+00,  3.6582e+00,
  

In [51]:
# Get logits from the last hidden state
logits = outputs1.logits  # Shape: (batch_size, sequence_length, vocab_size)

# To generate the next token, take the argmax over the vocabulary dimension
next_token_id = torch.argmax(logits[:, -1, :], dim=-1)  # Shape: (batch_size)

# Append the next token to the input_ids
generated_ids = torch.cat([input_ids1, next_token_id.unsqueeze(-1)], dim=-1)

# Decode the generated token to string
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("Generated Text:", generated_text)

AttributeError: 'BaseModelOutputWithPast' object has no attribute 'logits'

In [ ]:
decoded_output = tokenizer.decode(outputs1[0], skip_special_tokens=True)
print(decoded_output)

In [46]:
print(outputs1.last_hidden_state)
print(outputs2.last_hidden_state)

tensor([[[ 2.2305, -0.8589,  2.2832,  ..., -1.3838, -0.5322, -0.3289]]],
       device='cuda:0', dtype=torch.float16)
tensor([[[ 2.2305, -0.8589,  2.2832,  ..., -1.3838, -0.5322, -0.3289]]],
       device='cuda:0', dtype=torch.float16)


In [45]:
# Normalize embeddings
embedding1 = f.normalize(embedding1, p=2, dim=1)
embedding2 = f.normalize(embedding2, p=2, dim=1)

# Compute cosine similarity
cosine_sim = torch.mm(embedding1, embedding2.T)
print(f"Cosine similarity: {cosine_sim.item()}")

Cosine similarity: 1.0
